In [2]:
!nvidia-smi

Sun Aug 27 06:26:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   65C    P0    55W / 300W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True)

In [6]:
model_meso = AutoModelForCausalLM.from_pretrained(
    'mesolitica/llama-7b-hf-2048-fpf', 
    device_map="auto", 
    load_in_4bit=True,
)

[2023-08-27 06:26:48,775] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-27 06:26:49.136756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 06:26:49.805112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model_base = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-hf', 
    device_map="auto", 
    load_in_4bit=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

In [15]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False
    
stop_words = ["\n", "\nYou:", "You:"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze()[1:] for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [18]:
prompts = [
    'Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.',
    'Teruskan dialog di bawah. Tulis satu jawapan untuk watak "Assistant". Anda boleh bertanya kepadanya apa sahaja yang anda mahu dan ia akan melakukan yang terbaik untuk memberi anda maklumat yang tepat dan berkaitan.'
]

In [19]:
questions = [
    'siapa perdana menteri malaysia',
    'KWSP tu apa',
    'awat rafizi introduced vending machine?',
    'kerajaan madani banyak membantu tak?'
]

In [20]:
max_new_tokens = 500
temperature = 0.8
top_p = 0.95
top_k = 20
num_beams = 1
do_sample = True
repetition_penalty = 1.15

In [21]:
for p in prompts:
    for q in questions:
        query = f"""
{p}
You: {q}
Assistant:
        """
        query = query.strip()
        inputs = tokenizer(query, return_tensors='pt')
        input_ids = inputs['input_ids'].to(model_meso.device)

        with torch.no_grad():
            generation_output = model_meso.generate(
                input_ids=input_ids,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                num_beams=num_beams,
                do_sample=do_sample,
                stopping_criteria=stopping_criteria,
                repetition_penalty=repetition_penalty,
        )
        t = tokenizer.decode(generation_output.sequences[0])
        t = t.split('Assistant: ')[1]
        t = t.split('You:')[0]
        print(query)
        print(t)
        print()
        

Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: siapa perdana menteri malaysia
Assistant:
Prime Minister of Malaysia is Datuk Seri Anwar Ibrahim. He was sworn in on 24 November, 1993 as the fifth prime minister of Malaysia. The current Deputy Prime Minister is Datuk Seri Najib Razak who has been appointed by Prime Minister Datuk Seri Abdullah Ahmad Badawi in April, 2009 following the resignation of Deputy Prime Minister Datuk Seri Najib Tun Razak in March, 2009. Datuk Seri Najib Razak became the sixth prime minister of Malaysia when he was sworn in on the 3rd April 2009 at Istana Negara.</s>

Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: KWSP tu apa
Assistant:
The EPF is an organisation responsi

In [22]:
for p in prompts:
    for q in questions:
        query = f"""
{p}
You: {q}
Assistant:
        """
        query = query.strip()
        inputs = tokenizer(query, return_tensors='pt')
        input_ids = inputs['input_ids'].to(model_base.device)

        with torch.no_grad():
            generation_output = model_base.generate(
                input_ids=input_ids,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                num_beams=num_beams,
                do_sample=do_sample,
                stopping_criteria=stopping_criteria,
                repetition_penalty=repetition_penalty,
        )
        t = tokenizer.decode(generation_output.sequences[0])
        t = t.split('Assistant: ')[1]
        t = t.split('You:')[0]
        print(query)
        print(t)
        print()

Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: siapa perdana menteri malaysia
Assistant:
Malaysian Prime Minister is Najib Tun Razak, since 3 April 2009. His office is located at Perdanasana, Putrajaya (67104). The current Deputy Prime Minister of Malaysia is Muhyiddin Yassin. He was appointed on 8 September 2008 following the resignation of his predecessor Hishammuddin Hussein due to ill health. Both are members of Barisan Nasional or National Front political party.</s>

Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: KWSP tu apa
Assistant:
I'm not sure what that means, but if your question is about KWSP (Kumpulan Wang Simpanan), then here are some of their services and rates. 1) Savings Account